## 1. 데이터 불러오기


In [ ]:
# library 불러오기 
import pandas as pd
import seaborn as sns
import plotly.express as px
pd.set_option('display.max_columns', 30) # 최대 출력 변수 개수 수정

In [ ]:
# 예제 데이터
df_or = pd.read_excel('./data/Online Retail.xlsx')

In [ ]:
# 예제 데이터 처리
df_or['InvoiceDate2'] = df_or['InvoiceDate'].dt.normalize()
df_or['InvoiceMonth'] = df_or['InvoiceDate'].dt.month
df_or['InvoiceYear'] = df_or['InvoiceDate'].dt.year
df_or['InvoiceYM'] = df_or['InvoiceDate'].dt.strftime('`%y.%m.')
# df_or['InvoiceYM'] = df_or['InvoiceDate'].dt.to_period('M')
df_or['Amount'] = df_or['Quantity']*df_or['UnitPrice']

In [ ]:
df_or.tail(3)

In [ ]:
# 실습 데이터
df_ss = pd.read_excel('./data/Global Superstore 2018.xlsx')
df_ss['Order YM'] = df_ss['Order Date'].dt.to_period('M').dt.to_timestamp()
df_ss.tail(3)

<br>

## 2. 그룹 집계값의 계산과 그래프 작성  


* 참고: [seaborn](https://seaborn.pydata.org/examples/)

### 2.1. groupby() 등을 활용한 집계값 계산

In [ ]:
# 전체 합계, 평균 등 계산
df_or['Amount'].sum(), df_or['Amount'].mean(), df_or['Amount'].median(), df_or['Amount'].count()

In [ ]:
# agg()의 활용
df_or['Amount'].agg(['count', 'sum', 'mean'])

In [ ]:
# agg()의 활용(2): 매출액의 건수/합계, 평균, 수량의 합계
df_or.agg({'Amount':['count', 'sum', 'mean'], 'Quantity':['sum']})

In [ ]:
# 그룹별 집계값의 계산: 제품별 주문건수, 매출액, 수량
df_or_agg1 = df_or.groupby('StockCode', as_index=False).agg({'Amount':['count', 'sum'], 'Quantity':['sum']})
df_or_agg1

In [ ]:
# 그룹별 집계값의 계산: 고객별 매출액
df_or_agg2 = df_or.groupby('CustomerID', as_index=False)['Amount'].agg(['count', 'sum'])
df_or_agg2

In [ ]:
# 추가 정보 확인: sum이 0인 경우
df_or[df_or['CustomerID']==12346]

In [ ]:
# 추가 정보 확인: 매출액 기준 상위 10명 고객
df_or_agg2.nlargest(10, 'sum')

In [ ]:
# 추가 정보 확인: 매출액 기준 상위 10명 고객의 주요 구매 상품
target_cust = df_or_agg2.nlargest(10, 'sum')['CustomerID']
c1 = df_or['CustomerID'].isin(target_cust)
df_or_agg3 = df_or[c1].groupby('StockCode', as_index=False).agg({'Amount':['sum'], 'Quantity':['sum']})
df_or_agg3.sort_values(('Amount', 'sum'), ascending=False)

<br>

### [실습] df_ss에서 집계값 계산하기


1. 'Sales'의 전체 합계 계산
2. 'Customer ID'별 'Sales'의 합계 계산
3. 'Product ID'별 'Sales'의 합계와 'Profit'의 합계를 한번에 계산


In [ ]:
df_ss.head()

<br>

### 2.2. 히스토그램과 막대그래프 작성

In [ ]:
# 집계 데이터 생성: 제품별 매출액
df_or_agg1 = df_or.groupby('StockCode', as_index=False)['Amount'].sum()
df_or_agg1

In [ ]:
#  히스토그램을 활용한 'Amount'의 분포 확인
sns.histplot(data=df_or_agg1, 
             x='Amount')

In [ ]:
#  일부 관측치를 활용한 히스토그램을 활용한 'Amount'의 분포 확인
c1 = df_or_agg1['Amount'].between(0, 20000)
sns.histplot(data=df_or_agg1[c1], 
             x='Amount')

In [ ]:
# 일부 제품, 매출액의 막대그래프
sns.barplot(data=df_or_agg1.nlargest(20, 'Amount'),
            y='StockCode',
            x='Amount')

<br>

### [실습] df_ss에서 집계값을 그래프로 표현하기


1. 'Customer ID'별 'Sales'의 합계 계산
2. 1.에서 'Sales'의 분포를 히스토그램으로 확인하기
3. 1.의 'Sales'의 합계 기준 상위 20개 고객을 선택하고 막대그래프 그리기


<br>


### 2.3. 여러 그룹변수를 활용한 집계값 계산

In [ ]:
# 월, 국가별 매출액 추이
df_or_agg4= df_or.groupby(['InvoiceYM', 'Country'], as_index=False)['Amount'].sum()
df_or_agg4

In [ ]:
# 선그래프 활용
sns.lineplot(data=df_or_agg4,
             x='InvoiceYM',
             y='Amount',
             hue='Country')

In [ ]:
# 일부 관측치 활용 
c1 = df_or_agg4['Country'].isin(['Spain','France','Germany'])
sns.lineplot(data=df_or_agg4[c1],
             x='InvoiceYM',
             y='Amount',
             hue='Country')

<br>

### [실습] df_ss에서 집계값을 그래프로 표현하기(2)

1. 'Order YM', 'Category'별 'Sales'의 합계 계산
2. 1.에서 'Order YM'에 따른 'Sales' 합계의 추이를 'Category'별로 색을 다르게 한 선그래프로 표현하기


### 2.4. 집계값의 파생 변수 생성

In [ ]:
# 연, 국가별 매출액 추이
df_or_agg4= df_or.groupby(['InvoiceYM', 'Country'], as_index=False)['Amount'].sum()
df_or_agg4

In [ ]:
# 국가별 전월 매출액 변수 추가
df_or_agg4['LAmount'] = df_or_agg4.groupby('Country')['Amount'].shift()
df_or_agg4

In [ ]:
# 국가별 전월 대비 매출액 증감률 변수 추가
df_or_agg4['RAmount'] = (df_or_agg4['Amount'] - df_or_agg4['LAmount'])/df_or_agg4['LAmount']*100
df_or_agg4

In [ ]:
# 일부 관측치 막대그래프 
c1 = df_or_agg4['InvoiceYM'] == '`11.12.'
sns.barplot(data=df_or_agg4[c1], 
            y='Country',
            x='RAmount')

<br>

### [실습] df_ss에서 집계값을 그래프로 표현하기(3)

1. 'Order YM', 'Category'별 'Sales'의 합계 계산
2. 1.에서 'Category'별 전월대비 'Sales'의 증감률을 계산
3. 2.의 'Order YM'과 'Sales의 증감률'을 'Category'별로 색을 다르게 한 선그래프로 작성


<br>


## 3. 피벗테이블과 열지도

### 3.1. 피벗테이블 생성 

pandas의 *pivot_table()* 을 활용하면 Excel의 피벗테이블과 동일한 표 형태의 집계 가능

In [ ]:
# pivot_table()을 활용한 교차표 작성
    ## values : 값 변수
    ## index  : 행 그룹변수
    ## columns: 열 그룹변수 
    ## aggfunc: 집계 함수
pt1 = df_or.pivot_table(values='Amount', index='Country', columns='InvoiceYM', aggfunc='sum')
pt1

In [ ]:
# margin(행/열/전체 합계 )
df_or.pivot_table(values='Amount', index='Country', columns='InvoiceYM', aggfunc='sum', margins=True)

In [ ]:
# 열지도 시각화
# sns.heatmap(pt1, cmap='Blues', annot=True, fmt='.2f')
sns.heatmap(pt1, cmap='Blues')

<br>

### [실습] df_ss에서 집계값을 그래프로 표현하기(3)

1. 'Market', 'Category'별 'Profit'의 합계 피벗테이블 생성
2. 1.의 피벗테이블을 열지도로 그리기

## 4. 반응형 그래프의 활용



**plotly**를 활용해서 반응형 그래프를 그려서 탐색적 데이터 분석(EDA) 과정에서 유용하게 활용 가능

* 참고: [Plotly](https://plotly.com/python/)

### 4.1. 반응형 산점도 예제

In [ ]:
# 집계값 계산: 상품별 주문건수, 주문수량 합계, 평균 단가, 주문금액 합계
df_or_agg6 = df_or.groupby(['StockCode', 'Description'], as_index=False).agg({'InvoiceNo':'count', 'Quantity':'sum', 'UnitPrice':'mean', 'Amount':'sum'})
df_or_agg6

In [ ]:
# plotly 그래프가 나오지 않을 때 실행
# import plotly.offline as pyo
# pyo.init_notebook_mode()

In [ ]:
# 반응형 산점도 작성
px.scatter(data_frame=df_or_agg6.nlargest(100, 'Quantity'),
           x='Quantity',
           y='Amount', 
           size='UnitPrice', 
           color='InvoiceNo')

In [ ]:
# 반응형 산점도 작성
px.scatter(data_frame=df_or_agg6.nlargest(100, 'Quantity'),
           x='Quantity',
           y='Amount', 
           size='UnitPrice', 
           color='InvoiceNo', 
           
           width=600, 
           template='simple_white',
           hover_name='Description',
           hover_data=['StockCode'], 
           color_continuous_scale='Greens')

### 4.2. 일별 집계 중심 선그래프

In [ ]:
df_or_agg7 = df_or.groupby(['InvoiceDate2', 'Country'], as_index=False)['Amount'].sum()
df_or_agg7

In [ ]:
px.line(data_frame=df_or_agg7,
        x='InvoiceDate2',
        y='Amount',
        color='Country')

### 4.3. 막대그래프

In [ ]:
df_or_agg8 = df_or.groupby(['InvoiceYM', 'Country'], as_index=False)['Amount'].sum()
df_or_agg8

In [ ]:
# 비율 변수 추가
df_or_agg8['RAmount'] = df_or_agg8.groupby('InvoiceYM')['Amount'].apply(lambda x: x/sum(x)).to_list()
df_or_agg8

In [ ]:
px.bar(data_frame=df_or_agg8,
       x='InvoiceYM',
       y='RAmount',
       color='Country')

<br>

### [실습] df_ss로 반응형 그래프 그리기

1. groupby()를 활용해 'Market', 'Category'별 'Profit'의 합계를 계산하고 저장하기
2. 1.의 결과물을 x축을 'Market', y축을 'Profit'의 합계, color와 text를 'Category'로 지정한 막대그래프로 표현하기
3. groupby()를 활용해 'Order Date', 'Sub-Category'별  'Sales'의 합계를 계산하고 선그래프로 그리기

In [ ]:
df_ss.head()

### End of script